In [ ]:
%pip install backtrader

In [ ]:
df

In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the data

# Load the data (tab-separated file)
df = pd.read_csv('../csv/MSFT_M1_202402291729_202503272108.csv', sep='\t')  # Use tab as the separator
df['Datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])  # Use the correct column names
df.set_index('Datetime', inplace=True)
df.drop(columns=['<DATE>', '<TIME>'], inplace=True)  # Drop the original columns
# Rename columns to match what backtrader expects (optional, if needed)
df.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICKVOL>': 'Volume'}, inplace=True)

# Custom Hull Moving Average Indicator
class HullMovingAverage(bt.Indicator):
    lines = ('hma',)
    params = (('period', 9),)  # Default HMA period

    def __init__(self):
        # Step 1: Calculate WMA(n/2) and WMA(n)
        wma_n = bt.indicators.WeightedMovingAverage(self.data, period=self.params.period)
        wma_n2 = bt.indicators.WeightedMovingAverage(self.data, period=int(self.params.period / 2))
        
        # Step 2: 2 * WMA(n/2) - WMA(n)
        raw_hma = 2 * wma_n2 - wma_n
        
        # Step 3: WMA of the result with period sqrt(n)
        self.lines.hma = bt.indicators.WeightedMovingAverage(raw_hma, period=int(np.sqrt(self.params.period)))

# Define the strategy
class HMACrossoverStrategy(bt.Strategy):
    params = (
        ('fast_period', 9),   # Fast HMA period
        ('slow_period', 21),  # Slow HMA period
        ('rsi_period', 14),   # RSI period for confirmation
        ('rsi_overbought', 70),  # RSI overbought level
        ('rsi_oversold', 30),    # RSI oversold level
    )

    def __init__(self):
        # Define the fast and slow HMAs
        self.fast_hma = HullMovingAverage(self.data.close, period=self.params.fast_period)
        self.slow_hma = HullMovingAverage(self.data.close, period=self.params.slow_period)
        self.crossover = bt.indicators.CrossOver(self.fast_hma, self.slow_hma)  # 1 for up, -1 for down
        
        # Add RSI for confirmation
        self.rsi = bt.indicators.RSI(self.data.close, period=self.params.rsi_period)

    def next(self):
        # Buy signal: Fast HMA crosses above Slow HMA and RSI is not overbought
        if self.crossover > 0 and self.rsi < self.params.rsi_overbought:
            if not self.position:
                self.buy()
        
        # Sell signal: Fast HMA crosses below Slow HMA and RSI is not oversold
        elif self.crossover < 0 and self.rsi > self.params.rsi_oversold:
            if self.position:
                self.sell()

# Set up the backtest
cerebro = bt.Cerebro()
cerebro.addstrategy(HMACrossoverStrategy)

# Add the data to cerebro
data = bt.feeds.PandasData(dataname=df)
cerebro.adddata(data)

# Set initial cash and commission
cerebro.broker.setcash(10000.0)  # Starting capital: $10,000
cerebro.broker.setcommission(commission=0.001)  # 0.1% commission per trade

# Add analyzers to track performance
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

# Run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Print performance metrics
sharpe = results[0].analyzers.sharpe.get_analysis()
drawdown = results[0].analyzers.drawdown.get_analysis()
trades = results[0].analyzers.trades.get_analysis()

print(f"Sharpe Ratio: {sharpe['sharperatio']:.2f}")
print(f"Max Drawdown: {drawdown['max']['drawdown']:.2f}%")
print(f"Total Trades: {trades['total']['total']}")
print(f"Winning Trades: {trades['won']['total']}")
print(f"Losing Trades: {trades['lost']['total']}")

# Plot the results
cerebro.plot()
plt.show()